# Predicting Movie Revenue

## Features  

**Variable to Predict:** Revenue


- Budget  
- Runtime  
- Genre  
- Release Date  
 - Season  
 - Holiday   
- Production Companies  
 - Number of companies involved  
 - Number of movies a company has made  
- Crew
 - Number of crewmembers  
- Cast
 - Gender Ratio for top 10/25
 - Number of previous movies top 10 cast have been in
- Collection
 - Is sequel  
